# <center> <h1>AB testing - exercices </h1> </center>

<img src="../images/ab_testing.png" width="200">

# Pourquoi faire un AB testing ?
Tester une différence de résultat en fonction d'un paramètre à deux modalités.
Exemple : taux de conversion en fonction du modèle de recommandation choisi (modèle A vs modèle B)

Warning : Comparer juste des moyennes ne permet pas de conclure ! Même si le résultat semble évident.

In [ ]:
get_ipython().magic(u'matplotlib inline')
%run -i ../utils/credentials.py
%run -i ../utils/imports.py
%run -i ../utils/plots.py
%run -i ../utils/stats.py

# Question 1

Explorer pour différents niveaux de risque les valeurs associées pour une loi de student avec 20 degrés de liberté.

In [ ]:
loi = scs.t(20)
p=interactive(plot_ppf_loi(loi), p=[0.5,0.6, 0.8, 0.9, 0.95, 0.975, 0.99])
display(p)

# Question 2

Un risque de première espèce à 5% et de seconde espèce à 20% sont-il compatibles avec un effet minimum détectable de 2 et un écart-type de 1 ?

In [ ]:
loi1 = scs.norm(0,1)
loi2 = scs.norm(2,1)

p=interactive(plot_cdf_2_lois(loi1, loi2,annoted=True), t=np.linspace(0.1, 3, 20))
display(p)

__Réponse__

Non car sous ces conditions (mde et var) il y a des seuils pour lesquels les deux types de risques sont supérieurs à ceux fixés dans l'énoncé. 

# En pratique comment ça se passe ?

## Question 3

Déterminer la taille d'un échantillon pour un AB testing ayant les caractéristiques suivantes : 
* un risque de première espèce de 5%
* un risque de deuxième espèce de 20% (i.e la puissance du test est de 80%)
* un taux de clics de base de 5%
* une différence détectable minimale de 5
* un écart-type de 40

__Réponse__

In [ ]:
risk1 = 0.05
risk2 = 0.2
mde = 5
var = 40**2

In [ ]:
sample_size = size_sample_AB_test(risk1 = risk1, risk2 = risk2, var = var, mde = mde)
sample_size = int(sample_size)
sample_size

## Question 4

Tester si les moyennes des vecteurs A et B sont égales ou non (test unilatérale : l'hypothèse alternative est que la moyenne du groupe B est supérieure)

In [ ]:
x_A = np.random.normal(500, np.sqrt(var) ,size=sample_size)
x_B = np.random.normal(500 + mde, np.sqrt(var) , size=sample_size)

__Réponse__
On accepte l'égalité si notre test renvoie "True"

In [ ]:
test_H0(x_A, x_B, risk1)

# Question 5

Réaliser une simulation pour vérifier si les risques associés au test sont bien ceux souhaités.

__Réponse__

In [ ]:
results_test_simu = []
mean_A = 500
mean_B = 500 + mde

for ii in range(0,10000):
    
    # loi normale centrée en mean_A avec un écart-type de np.sqrt(var)
    x_A = np.random.normal(mean_A, np.sqrt(var) ,size=sample_size)
    
    # loi normale centrée en mean_B avec un écart-type de np.sqrt(var)
    x_B = np.random.normal(mean_B, np.sqrt(var) , size=sample_size)
    results_test_simu.append(test_H0(x_A, x_B, risk1)[0])

"Calcul par simulation de la puissance du test : {}%".format(int((1 - np.mean(results_test_simu))*100))

In [ ]:
results_test_simu = []
mean_A = 500
mean_B = 500

for ii in range(0,10000):
   # loi normale centrée en mean_A avec un écart-type de np.sqrt(var)
    x_A = np.random.normal(mean_A, np.sqrt(var) ,size=sample_size)
    
    # loi normale centrée en mean_B avec un écart-type de np.sqrt(var)
    x_B = np.random.normal(mean_B, np.sqrt(var) , size=sample_size)
    
    results_test_simu.append(test_H0(x_A, x_B, risk1)[0])

"Calcul par simulation de l'erreur de première espèce du test : {}%".format(int((1 - np.mean(results_test_simu))*100))

# Question 6

Relancer le test sur les données suivantes. Si le test permet de rejeter l'hypothèse nulle présenter les résultats au métier. Sinon relancez le test ;)

In [ ]:
x_A = np.random.normal(500, np.sqrt(var) ,size=sample_size)
x_B = np.random.normal(500 + mde, np.sqrt(var) , size=sample_size)

__Réponse__

In [ ]:
test_H0(x_A, x_B, risk1)

In [ ]:
confidence_interval_diff(x_A, x_B)

# Un problème complet : use-case de type multi-armed bandit

### Enoncé

On envisage de donner trois types de traitement à des personnes atteintes d'une maladie mortelle. La fonction de réponse __play_multi_armed_bandit__ permet d'obtenir la situation du patient après 6 mois de traitement en fonction du traitement qu'il a reçu. Le traitement à lui seul ne permet pas de prédire parfaitement l'état du patient, il y a donc une part aléatoire qui joue sur la situation finale de chacun des individus. 

L'exercice est de construire un algorithme permettant de choisir parmis les trois traitements "A", "B" et "C" pour chacun des 1000 patients de l'étude en maximisant le plus possible le nombre de patients en vie après 6 mois de traitement.

Comparer les résultats de cet algorithme avec ceux d'un algorithme de choix purement random.

Interpréter les résultats.

__NB__ : cet exercice est purement théorique et n'a d'autre objectif que de faire comprendre les qualités et défauts de stratégies différentes. Il n'est ici nullement discuté des enjeux moraux et médicaux. De plus, il est possible et même probable que de meilleures solutions existent pour résoudre ce problème.

In [ ]:
params = {"A": 0.5,
          "B": 0.6,
          "C": 0.2}

nb_steps = 2500

sep1 = "\n_____________________________"
sep2 = "_____________________________\n"

In [ ]:
def play_multi_armed_bandit(chosen_arm, params = params):
    for ii in params.keys():
        if chosen_arm == ii:
            value = np.random.binomial(1, params[ii], size=1)[0]
    return value

In [ ]:
def get_UCB(results, t):
    UCB = results.copy()
    for arm in results.keys():
        UCB[arm] = np.mean(results[arm]) + np.sqrt(2*np.log(t)/len(results[arm]))
    return UCB

In [ ]:
def solve_problem(nb_step, algo):
    results = {"A": [play_multi_armed_bandit("A")],
               "B": [play_multi_armed_bandit("B")],
               "C": [play_multi_armed_bandit("C")]}
    t = 0

    for ii in range (nb_steps - len(results)):
        
        t +=1
        
        if algo=="UCB":
            UCB = get_UCB(results,t)
            chosen_arm = max(UCB.items(), key=operator.itemgetter(1))[0]
        elif algo == "random":
            chosen_arm = random.choice(["A","B","C"])
        
        results[chosen_arm] += [play_multi_armed_bandit(chosen_arm)]

    return results

In [ ]:
results_UCB = solve_problem(nb_steps, "UCB")
results_random = solve_problem(nb_steps, "random")

In [ ]:
def evaluate_results(results):

    print(sep1)
    for ii in results.keys():
        
        print("moyenne pour le bras {} : {}".format(ii,np.mean(results[ii])))
    print(sep2)


    print(sep1)
    for ii in results.keys():
        print("nombre de coups pour le bras {} : {}".format(ii,len(results[ii])))
    print(sep2)
        
    length = 0
    score = 0
    for ii in results.keys():
        length += len(results[ii]) 
        score  += params[ii]*len(results[ii])

    for ii in results:
        regret = 0.6*length - score
    print(sep1)
    print("le regret vaut : {}".format(regret))
    print(sep2)
    
    return score, regret

In [ ]:
score_UCB, regret_UCB = evaluate_results(results_UCB)

Par rapport à la stratégie optimale (qui n'est réalisable qu'en cas d'omniscience), l'algorithme UCB a un regret de N. Cela signifie qu'en moyenne sur 1000 visiteurs, un choix  obtimal aurait permis de faire convertir N personnes de plus.

In [ ]:
score_random, regret_random = evaluate_results(results_random)

L'algorithme UCB est bien meilleure qu'une décision randomisée qui elle a un regret supérieur, la différence est une estimation du nombre de personnes convertir en plus par la méthode UCB.

#### Calculer l'intervalle de confiance de la différence entre les options A et B

In [ ]:
confidence_interval_diff(results_UCB["A"], results_UCB["B"]) # x_B - x_A

# Get more on my github <img src="./images/github.png" width="100">
https://github.com/JohanJublancPerso/datascience_statistics_tools

In [ ]:
# jupyter nbconvert --to slides AB_testing_exercices.ipynb --post serve